In [121]:
from git import Repo, log
import os
import subprocess
import csv
import re

In [138]:
# Extract commits that modified IaC scripts
organizations = ['Mirantis', 'Mozilla', 'Opendev', 'Wikimedia']
extensions = ['.pp', '.erb', '.epp', '.hiera']

for organization in organizations:
    repositories = os.listdir('./%s'%organization)
    for repo in repositories:
        iac_commits = []
        commit_messages = []
        path = './%s/%s'%(organization, repo)
        
        # Run the 'git log' command to retrieve commit logs
        try:
            for extension in extensions:
                # Here we list the extensions
                git_log_output = subprocess.check_output(['git', 'log','--oneline', '--', f'*{extension}'],
                                                         cwd=path, text=True)
                iac_commits.extend(git_log_output.strip().split('\n'))
        except subprocess.CalledProcessError as e:
            print(f"Error running 'git log': {e}")
        print(iac_commits)
        break
            
        last_id = ''
        messages = ''
        for line in iac_commits:
            # If this is updated, we should do the update
            if 'commit' == line[:6]:
                current_id = line.split()[1]
                # Try to find the matching issue number
                git_show_command = ['git', 'show', '--format=%B', '%s'%last_id]
                try:
                    commit_message = subprocess.check_output(git_show_command, cwd=path, text=True).strip()
                    pattern = r'#(\d+)'
                    matches = re.findall(pattern, commit_message)
                except subprocess.CalledProcessError as e:
                    print(f"Error running 'git show': {e}")
    
                if last_id != '':
                    if matches:
                        commit_messages.append({'ID': last_id, "Messages": messages, 'Unique Identifier': matches[0]})
                    else:
                        commit_messages.append({'ID': last_id, "Messages": messages, 'Unique Identifier': ''})
                last_id = current_id
                messages = ''
                continue
            # Skip the author, date, empty things, and authors
            if 'Author' == line[:6] or 'Date' == line[:4] or line == '' or 'Signed-off-by' in line:
                continue
            else:
                messages += line.replace('  ', '')
            
        # Update for the last message
        if current_id != '':
            # Try to find the matching issue number
            git_show_command = ['git', 'show', '--format=%B', '%s'%current_id]
            try:
                commit_message = subprocess.check_output(git_show_command, cwd=path, text=True).strip()
                pattern = r'#(\d+)'
                matches = re.findall(pattern, commit_message)
            except subprocess.CalledProcessError as e:
                print(f"Error running 'git show': {e}")
            if matches:
                commit_messages.append({'ID': current_id, "Messages": messages, 'Unique Identifier': matches[0]})
            else:
                commit_messages.append({'ID': current_id, "Messages": messages, 'Unique Identifier': ''})
        # Save in a csv file
        with open('./%s/%s_extracted_messages.csv'%(organization, repo), 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['ID', 'Messages', 'Unique Identifier'])
            writer.writeheader()
            writer.writerows(commit_messages)

['4d7efd4a Make puppet run on the new_db', '3fdea064 Merge pull request #277 from pigmej/pigmej-fix-158', '9f037ffa Added update action to nova_compute_libvirt_puppet', 'f5ab0fba Add update action for nova_compute', '1618caf9 Run and remove are the same', '9878adb9 Run and remove are the same', '17f80b4a Do not use hardcoded db port', '43329c8b Add and fix update actions. changing db_port should work now', "8c1dc662 Merge branch 'master' into librarian", '8da98c3c Copy action run to update', 'e077a157 Use default token backend. KVS seems to be broken', '4a3add5f Merge pull request #244 from bogdando/node_network_puppet', '19366f66 Add ceph_mon resource and example', '9751b879 Workaround l23network module issue', '289f022b Update node_network resource', '3d203c97 Add node_network puppet resource', '4dbfd63c Fix nova_generic_service resource', '94979f8d Add nova generic service resource', '8a5090ff Del duplicated puppet parameter', '23899f01 Fix ml2 agent resource and compostion', '2c185

In [141]:
git_show_command = ['git', 'show', '--format=%B', '%s'%'21112df4']
try:
    commit_message = subprocess.check_output(git_show_command, cwd=path, text=True).strip()
    pattern = r'#(\d+)'
    matches = re.findall(pattern, commit_message)
except subprocess.CalledProcessError as e:
    print(f"Error running 'git show': {e}")
print(matches)

[]
